In [1]:
#https://medium.com/datadriveninvestor/choosing-the-best-algorithm-for-your-classification-model-7c632c78f38f
#https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
#https://www.dataquest.io/blog/sci-kit-learn-tutorial/
#https://datascience.stackexchange.com/questions/33256/how-to-apply-machine-learning-model-to-new-dataset

In [2]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt


In [3]:
path = Path('/Users/bbabu/fastai/DataSamples')

In [4]:
#For V3
data = pd.read_csv(path/'Output-5v4.csv', sep=',', header=0)


In [5]:
le = preprocessing.LabelEncoder()

#convert the categorical columns into numeric
data['GroupDN'] = le.fit_transform(data['GroupDN'])
data['Member'] = le.fit_transform(data['Member'])
data['BusinessGroup'] = le.fit_transform(data['BusinessGroup'])
data['Platform'] = le.fit_transform(data['Platform'])
#data['Title'] = le.fit_transform(data['Title'])
#data['Department'] = le.fit_transform(data['Department'])
data['Country'] = le.fit_transform(data['Country'])
#data['State'] = le.fit_transform(data['State'])
data['City'] = le.fit_transform(data['City'])
#display the initial records
#data.head()

In [6]:
#Create Dependent and Independent Datasets based on our Dependent #and Independent features
X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Country', 'City']]
y = data['Owner']
#Split the Data into Training and Testing sets with test size as #50%
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.50, shuffle=True)

In [7]:
#for clf in classifiers:
clf = GaussianNB(priors=[0.95, 0.05], var_smoothing=1e-01)
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy of %s is %s"%(clf, acc))
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix of %s is %s"%(clf, cm))

Accuracy of GaussianNB(priors=[0.95, 0.05], var_smoothing=0.1) is 0.9692307692307692
Confusion Matrix of GaussianNB(priors=[0.95, 0.05], var_smoothing=0.1) is [[567   0]
 [ 18   0]]


In [8]:
import pickle
#cart_model = tree.DecisionTreeClassifier() #may be
cart_model = GaussianNB(priors=[0.95, 0.05], var_smoothing=1e-01) #le-01 is final, this is better
#cart_model = GaussianNB(priors=None, var_smoothing=1e-01) #le-01 is final
#cart_model = ComplementNB(alpha=3.0, class_prior=None, fit_prior=True, norm=False) #do not use
#cart_model  = KNeighborsClassifier() #Do not use
#cart_model = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=6) #may be
#cart_model = xgboost.XGBClassifier() #Do not use
#cart_model = svm.SVC(kernel="linear", C=0.025, gamma=2) #Do not use
cart_model.fit(X_train, y_train)

GaussianNB(priors=[0.95, 0.05], var_smoothing=0.1)

In [9]:
# Save model to disk
filename = 'Final_Model.sav'
pickle.dump(cart_model, open(filename, 'wb'))

In [10]:
# Load model from disk and use it to make new predictions
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9692307692307692


In [11]:
# Load test dataset
#final_predict = np.loadtxt(path/'Output-6-Test-V3.csv', delimiter=",")
df_test = pd.read_csv(path/'Output-6-Test-V4.csv', sep=',', header=0)

In [12]:
le = preprocessing.LabelEncoder()

#convert the categorical columns into numeric
df_test['GroupDN'] = le.fit_transform(df_test['GroupDN'])
df_test['Member'] = le.fit_transform(df_test['Member'])
df_test['BusinessGroup'] = le.fit_transform(df_test['BusinessGroup'])
df_test['Platform'] = le.fit_transform(df_test['Platform'])
#df_test['Title'] = le.fit_transform(df_test['Title'])
#df_test['Department'] = le.fit_transform(df_test['Department'])
df_test['Country'] = le.fit_transform(df_test['Country'])
#df_test['State'] = le.fit_transform(df_test['State'])
df_test['City'] = le.fit_transform(df_test['City'])
#display the initial records
#df_test.head()

In [13]:
X_train = df_test
pred = cart_model.predict(X_train)
print(pred)

[1 1 0 0 ... 1 1 0 0]


In [14]:
#df_test = le.inverse_transform(df_test)
final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Owner': pred})
final_df.to_csv(path/'MLProcessed1-Sklearn-GB.csv', header=True, index=False)